# Урок 4
Эта демонстрация разбита на 3 ноутбука:

1. Свертки и пулинги.
2. **Даталоадеры.**
3. Задача классификации с использованием CNN.

## Про загрузку данных

Во всех предыдущих примерах мы всегда учили сеть на всех данных сразу.
Но это возможно не всегда.

Хороший пример - [IMDB-WIKI](https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/), урезанная версия которого весит порядка 8 Gb.

In [2]:
# Если работаете в colab, запустите команды ниже.
# Они скачают и распакуют датасет.
# Должна получиться папка imdb_crop.tar

!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
!tar xf imdb_crop.tar

--2025-02-03 11:20:30--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7012157440 (6.5G) [application/x-tar]
Saving to: ‘imdb_crop.tar.1’

imdb_crop.tar.1     100%[===================>]   6.53G  1.57MB/s    in 46m 58s 

2025-02-03 12:07:28 (2.37 MB/s) - ‘imdb_crop.tar.1’ saved [7012157440/7012157440]

imdb_crop/01/nm0000401_rm2195568896_1961-7-30_2000.jpg: Truncated tar archive
tar: Error exit delayed from previous errors.


In [ ]:
from scipy.io import loadmat

imdb_dat = loadmat("imdb_crop/imdb.mat")["imdb"][0][0]
imdb_paths = [f"imdb_crop/{path[0]}" for path in imdb_dat[2][0]]
imdb_genders = imdb_dat[3][0]
# 1 означает Male, 0 - Female
print("genders data:", imdb_genders)
print("path to imgs:", imdb_paths[0])

In [ ]:
import os

import tqdm

total_size = 0
for one_path in tqdm.tqdm(imdb_paths):
    total_size += os.path.getsize(one_path)

In [ ]:
# В гигабайтах
total_size / 2**10 / 2**10 / 2**10

Учтите, что во время обучения нам нужно примерно х2 памяти - на прямой и обратный проход.

Если учить все одним батчом, то будет 12+ Гб на видеокарте - такое уже не каждая GPU потянет.

Без батчей тут не обойтись. Пойдем таким путем:

- научимся загружать одну картинку в тензор;
- научимся объединять несколько картинок в батчи.

### Как загрузить одну картинку в тензор

In [ ]:
# Вариант 1 - использовать matplotlib
# С ним уже виделись ранее, когда работали с NotMNIST
import matplotlib.pyplot as plt

image = plt.imread(imdb_paths[0])
print(image.shape)
print(image)

In [ ]:
# Вариант 2 - использовать PIL
import numpy as np
from PIL import Image

# Image.open вернет специальный объект Image
image = Image.open(imdb_paths[0])
print(type(image))
# который легко конвертируется в numpy массив
img_array = np.array(image)
print(img_array.shape)
print(img_array)

In [ ]:
# Вариант 3 - cv2 (a.k.a. opencv-python)
import cv2

cv_image = cv2.imread(imdb_paths[0])
print(type(cv_image))
print(cv_image.shape)
print(cv_image)

Основные различия: opencv и PIL имеют более богатый набор для редактирования самого изображения, но эти две библиотеки нужно отдельно установить.

У opencv есть интеграция с `albumentations`, которую мы будем использовать, поэтому возьмем `cv2.imread`.

### Как объединить несколько картинок в батч
В PyTorch уже есть готовое решение того, как бить данные на батчи.
Для этих целей используется **Dataset** и **DataLoader**.

Но перед тем, как их использовать, нужно подчистить данные.


In [ ]:
# Первая проблема - в датасете не везде есть метки
np.count_nonzero(np.isnan(imdb_genders))
# Выкинем их

In [8]:
bad_indices = set(np.where(np.isnan(imdb_genders))[0])
imdb_paths = [x for i, x in enumerate(imdb_paths) if i not in bad_indices]
imdb_genders = [int(x) for i, x in enumerate(imdb_genders) if i not in bad_indices]
assert len(imdb_paths) == len(imdb_genders)

In [ ]:
# Вторая проблема - картинки имеют разный размер
print(cv2.imread(imdb_paths[0]).shape)
print(cv2.imread(imdb_paths[1]).shape)

Это плохо - нейросеть, которую мы дальше будем учить, не сумеет состыковать размерности.
Помимо этого, мы не сможем собрать батч - у тензора жестко фиксирована размерность каждого среза.

Поэтому придется привести все картинки к одному размеру!

In [ ]:
# Есть библиотека albumentations, в которой есть часто используемые операции над картинками.
# В частности, resize до фиксированной размерности

import albumentations as A

# Compose означает "примени все трансформации из списка"
# У нас трансформация одна, но в будущем их может стать больше
transforms = A.Compose([A.Resize(128, 128)])

# В albumentations аргументы надо передавать с именем, на выходе будет словарь.
# Передали по имени `image`, заберем тому же ключу.
result = transforms(image=plt.imread(imdb_paths[0]))["image"]
print(result.shape)
print(type(result))
# Получили нужную размерность

Данные почищены, идем разбивать на батчи.

**Dataset** - это класс для хранения данных.
Задача Dataset - уметь отдавать пользователю один элемент данных.
Для этого нужно определить методы `__getitem__` и `__len__`.

**DataLoader** - это класс, который умеет разрезать _Dataset_ на батчи.
Он умеет бить на батчи, перемешивать их и загружать батчи параллельно с процессом обучения.

Чтобы пользоваться _DataLoader_, нужно сначала обернуть данные в _Dataset_.

In [ ]:
from torch.utils.data import Dataset, DataLoader


class SimpleDataset(Dataset):
    def __init__(self):
        # В методе __init__ можете сделать что угодно.
        # Обычно здесь готовят переменные, которые помогут загрузить данные
        pass

    def __getitem__(self, index):
        # __getitem__ должен отдать то, что вы считаете одним элементом датасета.
        # Тип данных не ограничен.
        # В переменной index лежит номер элемента, который заказал пользователь.
        return 1

    def __len__(self):
        # __len__ должен вернуть количество элементов в датасете.
        # Это должно быть целым числом.
        return 1


simple_dataset = SimpleDataset()
print(len(simple_dataset))
print(simple_dataset[0])
print(simple_dataset[1])
print(simple_dataset[100500])

Перейдем к нашему IMDB Wiki и попробуем написать для него Dataset.

In [ ]:
import albumentations as A
import cv2
import numpy as np
import torch
from albumentations.pytorch import ToTensorV2
from scipy.io import loadmat
from torch.utils.data import Dataset


class ImdbWikiDataset(Dataset):
    def __init__(self, image_size: int = 128):
        # Из кодов выше
        imdb_dat = loadmat("imdb_crop/imdb.mat")["imdb"][0][0]
        imdb_paths = [f"imdb_crop/{path[0]}" for path in imdb_dat[2][0]]
        imdb_genders = imdb_dat[3][0]
        bad_indices = set(np.where(np.isnan(imdb_genders))[0])
        imdb_paths = [x for i, x in enumerate(imdb_paths) if i not in bad_indices]
        imdb_genders = [
            int(x) for i, x in enumerate(imdb_genders) if i not in bad_indices
        ]

        # Не будем читать картинки при создании датасета, чтобы сберечь ОЗУ.
        self.paths = imdb_paths
        self.labels = imdb_genders
        self.transforms = A.Compose(
            [
                # Подгонит под размер (128, 128)
                A.Resize(image_size, image_size),
                # A.HorizontalFlip(p=0.5),
                # Пиксели в отрезке [0; 255] - это uint8.
                # Переведем в отрезок [0.0; 1.0] - нейросети будет проще.
                A.ToFloat(max_value=255),
                # Поменяет (H, W, C) -> (C, H, W) и превратит в тензор PyTorch
                ToTensorV2(),
                # Для обогащения: будем переворачивать
            ]
        )
        assert len(self.paths) == len(self.labels)

    def __getitem__(self, index) -> tuple[torch.Tensor, int]:
        # Читать будем только одну картинку - и возвращать пару (тензор картинки, ее label)
        img_numpy = cv2.imread(self.paths[index])
        img_tensor = self.transforms(image=img_numpy)["image"]

        label = self.labels[index]
        return img_tensor, label

    def __len__(self):
        return len(self.paths)


dataset = ImdbWikiDataset()
# Распечатаем несколько элементов из датасета
# Выдаст пару (изображение, лейбл)
one_item = dataset[0]
print(one_item[0].shape)
print(one_item[1])
one_item = dataset[5]
print(one_item[0].shape)
print(one_item[1])

In [ ]:
# Разобьем на train/val/test
from torch.utils.data import random_split

# Для воспроизводимости создадим генератор случайности
# и зафиксируем ему seed.
seed = 0
generator = torch.Generator()
generator.manual_seed(seed)

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [0.8, 0.1, 0.1], generator=generator
)
len(train_dataset), len(val_dataset), len(test_dataset)

In [14]:
from torch.utils.data import DataLoader

# Обернем датасет в DataLoader, передав batch_size
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    # перемешать данные или нет
    shuffle=True,
    # если перемешать - озаботьтесь воспроизводимостью
    generator=generator,
    # В последнем батче может не набраться 32 элемента.
    # Этот флаг говорит, убрать такой батч или оставить.
    drop_last=True,
)

In [ ]:
# После этого можно итерироваться по DataLoader
# Одна итерация = один батч
for one_batch in train_loader:
    batch_of_images, batch_of_labels = one_batch
    print(type(batch_of_images))
    print(batch_of_images.shape)
    print(type(batch_of_labels))
    print(batch_of_labels.shape)
    break

Обратите внимание: датасет возвращал тензор размера (3, 128, 128) и одно число, а вот DataLoader уже возвращет `(batch_size, 4, 128, 128)` и вектор из лейблов размера 32.

Pytorch собрал все за нас в батч и состыковал объекты из датасета.